In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import config
from datasets.datasethandler import DatasetHandler
datasetHandler = DatasetHandler()

In [3]:
import torch
import pytorch_lightning as pl
from classification.models.SpectrogramCNN import SpectrogramCNNPLModule
from classification.models.DeepRecursiveCNN import DeepRecursiveCNNPLModule
from classification.models.CRNN import CRNNPLModule

from classification.models.SpectrogramCNN_8K import SpectrogramCNN_8KPLModule
from classification.models.AblationModel import AblationModelPLModule

In [13]:
from pytorch_lightning.callbacks import Callback
import os

class SaveCallback(Callback): 
    def __init__(self, model_name):
        super().__init__()
        self.model_name = model_name
        self.best_val_acc = None

    def on_epoch_end(self, trainer, pl_module):
        if not self.best_val_acc or pl_module.val_results_history[-1]["val_acc"] > self.best_val_acc:
            print("new best val acc", pl_module.val_results_history[-1]["val_acc"])
            self.best_val_acc = pl_module.val_results_history[-1]["val_acc"]
            hparam_info = str({'random_time_stretch': pl_module.hparams['random_time_stretch'],
                           'mel_scale': pl_module.hparams['mel_scale'],
                           'normalize_spectrogram': pl_module.hparams['normalize_spectrogram']}).replace(" ", "_")
            try:
                os.mkdir("best_models/" + hparam_info + "/")
            except:
                print("folder exists already")
                
            save_path = "best_models/" + hparam_info + "/" + self.model_name + str(self.best_val_acc) + ".p"
            pl_module.save(save_path)
            print("Saved checkpoint at epoch {} at \"{}\"".format((trainer.current_epoch + 1), save_path))

In [ ]:
import itertools

for combination in list(itertools.product([True,False], repeat=3)):
    
    hparams = {'batch_size': 32,
     'learning_rate': 0.001130722521874597,
     'p_dropout': 0.2126022956703681,
     'lr_decay': 0.9254289926146729,
     'n_hidden': 823,
     'weight_decay': 0.0011007407642843825,
     'random_time_stretch': combination[0],
     'mel_scale': combination[1],
     'normalize_spectrogram': combination[2]}

    model = AblationModelPLModule(hparams)
    model.prepare_data()
    datasetHandler.load(model, 'training')#, dataset_id=config.DATASET_CONTROL)
    datasetHandler.load(model, 'validation')#, dataset_id=config.DATASET_CONTROL)

    cb = SaveCallback("ablation_")
    trainer = pl.Trainer(
        max_epochs=50,
        #logger= loggers.TensorBoardLogger(config.LOG_DIR, name=type(model)._name_),
        gpus=1 if torch.cuda.is_available() else None,
        callbacks=[cb]
    )

    trainer.fit(model)

GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.

   | Name                 | Type              | Params
-------------------------------------------------------
0  | model                | AblationModel     | 182 K 
1  | model.convs          | Sequential        | 114 K 
2  | model.convs.0        | BatchNorm2d       | 2     
3  | model.convs.1        | Conv2d            | 1 K   
4  | model.convs.2        | BatchNorm2d       | 20    
5  | model.convs.3        | PReLU             | 1     
6  | model.convs.4        | MaxPool2d         | 0     
7  | model.convs.5        | Dropout           | 0     
8  | model.convs.6        | Conv2d            | 20 K  
9  | model.convs.7        | BatchNorm2d       | 40    
10 | model.convs.8        | PReLU             | 1     
11 | model.co

Val-Acc=0.018375815056312982


Val-Acc=0.8002371072910492
Train-Acc=0.816798418972332
new best val acc 0.8002371072910492
Saved model to "best_models/{'random_time_stretch':_True,_'mel_scale':_True,_'normalize_spectrogram':_True}/ablation_0.8002371072910492.p"
Saved checkpoint at epoch 1 at "best_models/{'random_time_stretch':_True,_'mel_scale':_True,_'normalize_spectrogram':_True}/ablation_0.8002371072910492.p"


Val-Acc=0.8026081802015412
Train-Acc=0.8424901185770751
new best val acc 0.8026081802015412
folder exists already
Saved model to "best_models/{'random_time_stretch':_True,_'mel_scale':_True,_'normalize_spectrogram':_True}/ablation_0.8026081802015412.p"
Saved checkpoint at epoch 2 at "best_models/{'random_time_stretch':_True,_'mel_scale':_True,_'normalize_spectrogram':_True}/ablation_0.8026081802015412.p"


Val-Acc=0.8280972139893301
Train-Acc=0.8557312252964426
new best val acc 0.8280972139893301
folder exists already
Saved model to "best_models/{'random_time_stretch':_True,_'mel_scale':_True,_'normalize_spectrogram':_True}/ablation_0.8280972139893301.p"
Saved checkpoint at epoch 3 at "best_models/{'random_time_stretch':_True,_'mel_scale':_True,_'normalize_spectrogram':_True}/ablation_0.8280972139893301.p"


...

Val-Acc=0.8358032009484292
Train-Acc=0.8766798418972332


Val-Acc=0.8654416123295792
Train-Acc=0.8780632411067194


Val-Acc=0.8612922347362182
Train-Acc=0.8808300395256917


Val-Acc=0.8648488441019562
Train-Acc=0.8784584980237155


Val-Acc=0.8500296384113811
Train-Acc=0.8766798418972332


In [ ]:
# model.save("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/SpectrogramCNN/vanilla.p")